In [ ]:
############################
# FrenchNews.csv post process to generate FrenchNewsDayConcat.csv
# The post process sample FrenchNews.csv at day to compare it with CAC40
############################

In [ ]:
#fichier CSV
import csv
import time
import re

import datetime as dt

#Nom fichier CSV
CSV_FILE_NAME_INPUT = '../input/french-financial-news/FrenchNews.csv'
CSV_FILE_NAME_OUTPUT_TEMP = 'temp.csv'
CSV_FILE_NAME_OUTPUT = 'FrenchNewsDayConcat.csv'

nbrLigneTotalCsvInput=0

#determination taille fichier
with open(CSV_FILE_NAME_INPUT, 'r', newline='',encoding = 'utf-8') as file:
    reader = csv.reader(file, delimiter=',')
        
    for row in reader:
        
        #Calcule du nombre de ligne total
        nbrLigneTotalCsvInput += 1
        
        #On lit la 1ère date
        if nbrLigneTotalCsvInput==2:
            firstDateCsvIn=row[3]
            
    lastDateCsvIn=row[3]        
            
            
    
    #Calcule du nombre de ligne total
    #nbrLigneTotalCsvInput = sum(1 for row in reader)  # fileObject is your csv.reader
    
    
    print("nbrLigneTotal=" + str(nbrLigneTotalCsvInput))
    print("\n")
    print("firstDateCsvIn=" + firstDateCsvIn)
    print("lastDateCsvIn=" + lastDateCsvIn)
    print("\n")


    # Exemple : 08.01.2021
    dateFormat = "%d.%m.%Y"

    # Date debut
    dateDTfirst = dt.datetime.strptime(firstDateCsvIn, dateFormat)
    print('dateDTfirst:', dateDTfirst.strftime(dateFormat))

    # Date fin
    dateDTlast = dt.datetime.strptime(lastDateCsvIn, dateFormat)
    print('dateDTlast:', dateDTlast.strftime(dateFormat))

    ########################
    #Force une date de debut
    ########################
    #dateDTfirst = dt.datetime(2017, 11, 1)
    
    

In [ ]:
############################
# Récuperation des cours de bourse
############################

'''
Librairies à installer : 

    Numpy
    Matplotlib
    Pandas
    Pandas-datareader (-> pip install pandas-datareader)
    BeautifulSoup4
    scikit-learn / sklearn

'''


import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web


In [ ]:
#style.use('ggplot')



In [ ]:
df_stocksValues = web.DataReader("^FCHI", 'yahoo', dateDTfirst, dateDTlast)  # CAC40

#with pd.option_context('display.max_rows',10):
#    print(df_stocksValues)
    
df_stocksValues


In [ ]:
#df_stocksValues[['High','Low','Open','Close','Adj Close']].plot(figsize=(15, 10))
df_stocksValues[['High','Low','Open','Close','Adj Close']].plot(figsize=(15, 10))

plt.show()

df_stocksValues[['Volume']].plot(figsize=(15, 10))
plt.show()

In [ ]:
print("Date de début : \n    " + str(df_stocksValues.index[0]))

In [ ]:
# on concatène le sentiment des news de chaque jours
    
start = time.time()

# open csv file in "write" mode     // encoding utf-8 pour éviter les erreurs d'encodage
with open(CSV_FILE_NAME_OUTPUT_TEMP, 'w', newline='',encoding = 'utf-8') as fileOutput:
    writerOutput = csv.writer(fileOutput, delimiter=',')
    
    
    # open csv file in "read" mode     // encoding utf-8 pour éviter les erreurs d'encodage    
    with open(CSV_FILE_NAME_INPUT, 'r', newline='',encoding = 'utf-8') as file:
        
        reader = csv.reader(file, delimiter=',')

        indexJour=0
        
        nbrLignesTraiter=0
        nbrDateTraiter=0
        
        rowWrite=[]
        titleSentimentSum=0
        textSentimentSum=0
        textURLSentimentSum=0
        
        nbrNewsJour=0
        nbrLignesTraiterJour=0
        
        for row in reader:  
            
            nbrLignesTraiter += 1  
            nbrNewsJour += 1
            
            #Initialisation 1ère date
            if nbrLignesTraiter == 1:
                rowWrite.append("Nbr Day")
                rowWrite.append("Date") 
                rowWrite.append("NbrNewsJour") 
                rowWrite.append("Day Sent Vader Title")
                rowWrite.append("Day Sent Vader Text")
                rowWrite.append("Day Sent Vader Text URL")
                
                listBourse = list(df_stocksValues.columns.values)   #["High","Low","Open","Close","Volume","Adj Close"]
                rowWrite.extend(listBourse)
                
                writerOutput.writerow(rowWrite) 
                rowWrite=[]
                #indexJour += 1
                
            #Les dates suivantes   
            else:
                titleSentimentSum += float(row[17])
                textSentimentSum += float(row[18])
                textURLSentimentSum += float(row[19])

                #Date du cours de bourse
                dateStocks = df_stocksValues.index[indexJour]

                
                # Date news
                dateFormat = "%d.%m.%Y"
                dateNews = dt.datetime.strptime(row[3], dateFormat)
                
                print("dateStocks = " + str(dateStocks))
                print("dateNews =   " + str(dateNews))
                
                #S'il y a plusieurs news dans un jours
                if dateStocks < dateNews:
                    
                    print("### DATES DIFFERANTES ### dateStocks < dateNews")
                    nbrDateTraiter += 1
  
                    
                    #Preparation ecriture dans le fichier csv
                    rowWrite.append(indexJour)
                    rowWrite.append(dateStocks)
                    rowWrite.append(nbrNewsJour)
                    rowWrite.append(titleSentimentSum/nbrNewsJour)
                    rowWrite.append(textSentimentSum/nbrNewsJour)
                    rowWrite.append(textURLSentimentSum/nbrNewsJour)
                    #rowWrite.append(df_stocksValues["Open"][indexJour])

                    listBourse = list(df_stocksValues.iloc[indexJour].values)   #["High","Low","Open","Close","Volume","Adj Close"]
                    rowWrite.extend(listBourse)                    
                    
                    #Ecriture dans le fichier csv        
                    writerOutput.writerow(rowWrite) 

                    indexJour += 1

                    nbrNewsJour=0
                    
                    rowWrite=[]
                    titleSentimentSum=0
                    textSentimentSum=0
                    textURLSentimentSum=0
                
                #S'il y a pas de news durant un jour (arrive rarement)
                if dateStocks > dateNews:
                    
                    print("\n\n$$$$ DATES DIFFERANTES $$$$ PAS DE NEWS CE JOUR\n\n")
                    
                    # Dans ce cas, le sentiment en neutre => 0
                    #titleSentimentSum=0
                    #textSentimentSum=0  
                    #textURLSentimentSum=0 
                    
                    #il faut forcer l'écriture du cours pour rattraper
                    
                    
                    
             
            

    

In [ ]:

#determination taille fichier de sortie pour voir si tout s'est bien passé
with open(CSV_FILE_NAME_OUTPUT_TEMP, 'r', newline='',encoding = 'utf-8') as file:
    reader = csv.reader(file, delimiter=',')
    
    #Calcule du nombre de ligne total
    nbrLigneTotalCsvOutput = sum(1 for row in reader)  # fileObject is your csv.reader  
    
    print("Nbr de ligne total : \n")
    print("   Fichier d'entrée = " + str(nbrLigneTotalCsvInput))
    print("   Fichier de sortie = " + str(nbrLigneTotalCsvOutput))
    print("   Données cours     = " + str(len(df_stocksValues)))
    print("\n")



In [ ]:
# Reouverture fichier pour affichage graphique
with open(CSV_FILE_NAME_OUTPUT_TEMP, 'r', newline='',encoding = 'utf-8') as file:

    reader = csv.reader(file, delimiter=',')

    for row in reader:  

        nbrLignesTraiter += 1  
        nbrNewsJour += 1


In [ ]:
# !conda install -c plotly plotly
import plotly.express as px

df = pd.read_csv(CSV_FILE_NAME_OUTPUT_TEMP)

fig = px.line(df, x = 'Date', y = ['Day Sent Vader Title','Day Sent Vader Text','Day Sent Vader Text URL'], title='Sentiment analyse')
fig.show()

fig = px.line(df, x = 'Date', y = ['NbrNewsJour'], title='Number news per day (The more we are far from the scraping date the less news we get)')
fig.show()




In [ ]:
# Moyennage sentiment analysis 
# Voir : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

df = pd.read_csv(CSV_FILE_NAME_OUTPUT_TEMP)

#Pour test le moyennage est fait sur 20 jours
df['Mean sent title'] = df['Day Sent Vader Title'].rolling(window=20,min_periods=0,center=True).mean()

df['Mean sent text'] = df['Day Sent Vader Text'].rolling(window=20,min_periods=0,center=True).mean()

df['Mean sent text URL'] = df['Day Sent Vader Text URL'].rolling(window=20,min_periods=0,center=True).mean()


# Window type Triangle
#df['Mean sent text Tri'] = df['Day Sent Vader Text'].rolling(window=20,min_periods=0,center=True,win_type='triang').mean()

#print(df.head())

df[['Day Sent Vader Text','Mean sent text']].plot(figsize=(15, 10))
plt.show()

#fig = px.line(df, x = 'Date', y = ['Day Sent Vader Title','Day Sent Vader Text'], title='')
#fig.show()



In [ ]:
df[['Mean sent title','Mean sent text','Mean sent text URL']].plot(figsize=(15, 10))
plt.show()

In [ ]:
# Superposition graph
# Voir : https://stackoverflow.com/questions/62853539/plotly-how-to-plot-on-secondary-y-axis-with-plotly-express

from plotly.subplots import make_subplots

#df = pd.read_csv(CSV_FILE_NAME_OUTPUT_TEMP)

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(df, x = 'Date', y = 'Open', title='')
fig2 = px.line(df, x = 'Date', y = ['Mean sent text','Mean sent title','Mean sent text URL'], title='')

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Time"
subfig.layout.yaxis.title="Open"
#subfig.layout.yaxis2.type="log"
subfig.layout.yaxis2.title="Sentiment"
# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()


#fig.show()

In [ ]:
#OPTIONNEL : Ajout des données moyennés dans un nouveau fichier csv

In [ ]:
#On
df

In [ ]:



df.to_csv(CSV_FILE_NAME_OUTPUT, index=False)